# SEAPOL Tutorial

---

This interactive tutorial takes you through the steps of how to run RadxRate. RadxRate builds upon the capabilities of RadxKdp and RadxPid by calculating hourly rain rates at each radar gate depending on the local PID category and polarimetric values. It shares parameters files with those two applications. RadxRate will calculate the rain rates with either raw or attenuated-corrected data; the user can choose the option they prefer. The three-dimensional rain rates produced by RadxRate can be then used in RadxQpe to estimate the rain rate closest to the surface, after accounting for beam blockage, noise, and clutter.

A visual comparison of RadxKdp, RadxPid, and RadxRate and their parameter files is shown below. Each application has its own main parameter file that defines variable names and specifies the paths to the parameter file for each relevant sub-process. For example, the main RadxRate parameter file links to the KDP, PID, and rate parameter files (the PID parameter file links to the PID thresholds file).

<div>
<img src="../../ams2023/images/radx_echo_description.png" width="600"/>
</div>

---

*Note: this tutorial is just one type of workflow to use RadxRate. There are several other workflows!*

---

## Tutorial Overview
### 1. Setup

#### Directory organization

The structure of the echo tutorial on JupyterHub is shown in the diagram below. The parent or base directory is "ams2023" and contains all of the notebooks, parameter files, and data for the workshop.

All of the materials for the demonstration today are in 'seapol' subfolders including:
* data/seapol/qc
* data/seapol/sounding
* params/seapol/RadxRate_Kdp_params
* params/seapol/RadxRate_main_params
* params/seapol/RadxRate_Pid_params
* params/seapol/RadxRate_rate_params
* params/seapol/Pid_thresholds.cband.shv.txt

*Note: The parameter files have already been modified to run straight out of the box.*

#### Download raw data and parameter files

Raw data files that will be downloaded:
* cfrad.20220729_080113.268_to_20220729_080615.642_SEAPOL_SUR.nc
* cfrad.20220729_080847.591_to_20220729_081136.053_SEAPOL_RHI.nc

We will also use sounding files generated from model data over the domain.
* 20220729.data
* 20220729.indx


### 2. Prepare data for analysis

The data have already been prepared for this tutorial by converting to cfradial and running a quality control algorithm.
We also preprocessed a model sounding file to use in the PID for these cases. There are other LROSE tutorials available at http://wiki.lrose.net/index.php/Main_Page to help preprocess data.


### 3. Run RadxRate

* Calculate KDP, run PID algorithm, estimate three-dimensional precipitation rates
    * RadxRate


### 4. Plot Radar variables

* Visualize results of RadxRate analysis using Py-ART

### 5. Explaination of hydrometeor classification and rain rates

* Visualize PID and rain rates

---

For this tutorial, the parameter files have already been created and populated with the appropriate parameters. However, the default parameter files can be saved using the following commands.

In [ ]:
# # create original parameter files
# !/usr/local/lrose/bin/Grib2toMdv -print_params > /home/jovyan/ams2023/params/user/Grib2toMdv_params_user
# !/usr/local/lrose/bin/Mdv2SoundingSpdb -print_params > /home/jovyan/ams2023/params/user/Mdv2SoundingSpdb_params_user
# !/usr/local/lrose/bin/RadxRate -print_params > /home/jovyan/ams2023/params/user/RadxRate_main_params_user
# !/usr/local/lrose/bin/RadxRate -print_params_kdp > /home/jovyan/ams2023/params/user/RadxRate_Kdp_params_user
# !/usr/local/lrose/bin/RadxRate -print_params_pid > /home/jovyan/ams2023/params/user/RadxRate_Pid_params_user
# !/usr/local/lrose/bin/RadxRate -print_params_rate > /home/jovyan/ams2023/params/user/RadxRate_rate_params_user

# to create new parameter files that use parameters from another file
# !/usr/local/lrose/bin/RadxRate -params /path/to/params -print_params > /home/jovyan/ams2023/params/user/RadxRate_main_params_user

# for command line options
# !/usr/local/lrose/bin/RadxRate -h


# 1. Setup
## Environment and packages

First, we import the required python packages to run this notebook. Most of the LROSE processing can be done with the os package and shell commands. At the end we will plot the output using Py-ART.

In [ ]:
import os
import pyart
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np

Next, we set up the directory structure to simplify our commands. If you are running this notebook at the LROSE workshop on JupyterHub, these paths go to the parent directory containing all the workshop resources and the LROSE binaries. 

**If you have downloaded this notebook, please modify BASE_DIR and LROSE_DIR to work on your personal machine.**

* BASE_DIR: the base directory containing the directories for the notebooks, parameter files
* DATA_DIR: the base directory containing the directories for the data
* RADAR_NAME: the name of the radar used in this tutorial
* LROSE_DIR: path to the LROSE installation 

Note that we are setting both the *system* directories for LROSE to use, *and* the paths in python for reading the data.

In [ ]:
os.environ['BASE_DIR'] = '/home/jovyan/ams2023/'
os.environ['DATA_DIR'] = '/home/jovyan/ams2023/data/seapol/qc/'

os.environ['RADAR_NAME'] = 'SEA-POL'
os.environ['LROSE_DIR'] = '/usr/local/lrose/bin'
base_dir = os.environ['BASE_DIR']
data_dir = '/home/jovyan/ams2023/data/seapol/'
radar_name = os.environ['RADAR_NAME']
!echo "Base directory: "$BASE_DIR
!echo "Radar name: "$RADAR_NAME
!echo "LROSE directory: "$LROSE_DIR

## Data download and directory set up

We need to set up the required data directories and download the radar data and GFS analysis to the JupyterHub. We delete any existing files and directories specific to this tutorial to ensure we're starting with clean directories and files.

In [ ]:
# ## make a directory for all the data files in the seapol tutorial (qc and sounding)
!rm -rf ${BASE_DIR}/data
!mkdir ${BASE_DIR}/data


# ## make a directory for all the data files in the seapol tutorial (qc and sounding)
!rm -rf ${BASE_DIR}/data/seapol
!mkdir ${BASE_DIR}/data/seapol

# ## make subdirectory within data for the sounding data
!rm -rf ${BASE_DIR}/data/seapol/sounding
!mkdir ${BASE_DIR}/data/seapol/sounding

# ## wget raw files from SEAPOL server
!wget https://seapol.colostate.edu/data/UH_tutorial/Sounding/20220729.data
!wget https://seapol.colostate.edu/data/UH_tutorial/Sounding/20220729.indx

!mv 20220729.data ${BASE_DIR}/data/seapol/sounding
!mv 20220729.indx ${BASE_DIR}/data/seapol/sounding

# ## move files to proper directory
!mv cfrad.20220729_080113.268_to_20220729_080615.642_SEAPOL_SUR.nc ${BASE_DIR}/data/seapol/qc
!mv cfrad.20220729_080847.591_to_20220729_081136.053_SEAPOL_RHI.nc ${BASE_DIR}/data/seapol/qc

# ## make subdirectory within data for sounding data
!rm -rf ${BASE_DIR}/data/seapol/qc
!mkdir ${BASE_DIR}/data/seapol/qc

# ## wget raw files from SEAPOL server
!wget https://seapol.colostate.edu/data/UH_tutorial/QCed/cfrad.20220729_080113.268_to_20220729_080615.642_SEAPOL_SUR.nc
!wget https://seapol.colostate.edu/data/UH_tutorial/QCed/cfrad.20220729_080847.591_to_20220729_081136.053_SEAPOL_RHI.nc
# !wget http://front.eol.ucar.edu/data/notebooks/ams2023/KMHX20180914_191822_V06.ar2v

# ## move files to proper directory
!mv cfrad.20220729_080113.268_to_20220729_080615.642_SEAPOL_SUR.nc ${BASE_DIR}/data/seapol/qc
!mv cfrad.20220729_080847.591_to_20220729_081136.053_SEAPOL_RHI.nc ${BASE_DIR}/data/seapol/qc


In [ ]:
surname = 'cfrad.20220729_080113.268_to_20220729_080615.642_SEAPOL_SUR.nc'
datadir = '/home/jovyan/data/seapol/qc/'
sur_file = f'{datadir}{surname}'

We can examine the raw radar file using RadxPrint. RadxPrint can read CfRadial and other raw radar formats supported by LROSE that can later be converted by RadxConvert. It will also provide information on radar variables and sweep information. For example, we can look at the variables using the following command, piping the output into the head command.

In [ ]:

# print out the first 50 lines of RadxPrint output
!${LROSE_DIR}/RadxPrint -f ${DATA_DIR}cfrad.20220729_080113.268_to_20220729_080615.642_SEAPOL_SUR.nc | head -50


# 2. Prepare data for analysis
### These files have already been converted to cfradial and quality controlled, so they are ready to go into RadxRate. Additionally, the Parameter files have been set up to work out of the box to calculate:
* Kdp
* PID
* Rain rates using polarimetric estimators

# 3. Setup steps complete, run RadxRate

With the radar data and point sounding, we can now run RadxRate. First, KDP is calculated. Then polarimetric radar data, calculated KDP, and sounding data are used to run PID, which determines the dominant hydrometeor at each range gate. Finally, the 3D field of hourly rain rates are calculated from polarimetric radar and PID data.

Running RadxRate requires one main parameter file on the command line. That parameter file contains  links to the KDP, PID, and rain rate coefficient parameter files. The RadxRate PID parameter file contains the link to the fuzzy logic PID thresholds file. Because there are so many parameters, we have provided them at the end of this notebook. Please refer to [this link](#RadxRad-parameters).

RadxRate requires the field names in the input CfRadial file, whether SNR and LDR are available, the method for KDP calculation, how to calculate the PID, how to calculate the precipitation rates (e.g., equations and coefficients), whether to use attenuation-correction fields, the variables to be written, and whether to censor non-weather gates. 

Here, *-params* provides the link to the main RadxRate parameter file, *-f* provides the link to the files we want to process, and *-outdir* indicates where RadxRate should write the final files.

In [ ]:
!${LROSE_DIR}/RadxRate -params ${BASE_DIR}params/seapol/RadxRate_main_params -f ${DATA_DIR}/*.nc -outdir ${BASE_DIR}/data/seapol


# 4. SEAPOL Squall line on 29 July 2022 in Yonaguni, Japan
# Plot the low-level PPI reflectivity and radial velocity.

To visualize the output in the notebook, we can use Py-ART. HawkEye is also great for visualizing data - a general parameter file can be found in the echo parameter files directory.

## Read the SEAPOL PPI / Survellience file in with PyART.

In [ ]:
# Read CfRadial file into radar object
inDir = data_dir+"/20220729/"
file = "cfrad.20220729_080113.268_to_20220729_080615.642_SEAPOL_SUR.nc"
#file = "cfrad.20220729_214909.013_to_20220729_215410.811_SEAPOL_SUR.nc"
sur_seapol = pyart.io.read_cfradial(inDir+file)

#if you want to see information that is in the file, including field names, uncomment the next line.
#rate_seapol.info('compact')


## Read the SEAPOL RHI vertical slice in with PyART.

In [ ]:
pfile = "cfrad.20220729_080847.591_to_20220729_081136.053_SEAPOL_RHI.nc"
#pfile = "cfrad.20220729_214806.187_to_20220729_214906.248_SEAPOL_RHI.nc"
rhi_seapol = pyart.io.read_cfradial(inDir+pfile)
#rhi_seapol.info('compact')


In [ ]:
#Define some bounds for plotting the CAPPI

surx_min= -80
surx_max = 80
sury_min = -80 
sury_max = 80

sur_sweep = 2

In [ ]:
## Zoom in and look at a lower level sweep.

# surx_min= 15
# surx_max = 55
# sury_min = -55 
# sury_max = -15

# sur_sweep = 4

In [ ]:
# Plot the low-level surviellence

displayRate = pyart.graph.RadarDisplay(sur_seapol)
figRate = plt.figure(1, (12, 6))

# DBZ (input)

axDbz = figRate.add_subplot(121)
displayRate.plot_ppi('DBZ_ATTEN_CORRECTED', sur_sweep, vmin=0, vmax=80.,
                    axislabels=("x(km)", "y(km)"),cmap='pyart_NWSRef',
                    colorbar_label="DBZ")
displayRate.set_limits(xlim=[surx_min,surx_max],ylim=[sury_min,sury_max])
displayRate.plot_range_rings([30, 60])#, 150, 200, 250])
displayRate.plot_cross_hair(2.)
#displayRate.set_limits(xlim=(-70,70),ylim=(-70,70))

# VEL(input)

axDbz = figRate.add_subplot(122)
displayRate.plot_ppi('VEL', sur_sweep, vmin=-20, vmax=20.,
                    axislabels=("x(km)", "y(km)"),cmap='pyart_NWSVel',
                    colorbar_label="VR (m/s)")
displayRate.set_limits(xlim=[surx_min,surx_max],ylim=[sury_min,sury_max])
displayRate.plot_range_rings([30, 60])#, 150, 200, 250])
displayRate.plot_cross_hair(2.)
#displayRate.set_limits(xlim=(-70,70),ylim=(-70,70))



figRate.tight_layout()

plt.show()


## Now let's look at some polarimetric data

In [ ]:
# Plot results of RadxRate

displayRate = pyart.graph.RadarDisplay(sur_seapol)
figRate = plt.figure(1, (12, 10))

# DBZ (input)

axDbz = figRate.add_subplot(221)
displayRate.plot_ppi('DBZ_ATTEN_CORRECTED', sur_sweep, vmin=0, vmax=80.,
                    axislabels=("x(km)", "y(km)"),cmap='pyart_NWSRef',
                    colorbar_label="DBZ")
displayRate.set_limits(xlim=[surx_min,surx_max],ylim=[sury_min,sury_max])
displayRate.plot_range_rings([30, 60])#, 150, 200, 250])
displayRate.plot_cross_hair(2.)
#displayRate.set_limits(xlim=(-70,70),ylim=(-70,70))

# KDP (computed)

axKdp = figRate.add_subplot(222)
displayRate.plot_ppi('KDP', sur_sweep, vmin=0, vmax=5.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="KDP (deg/km)",
    cmap="nipy_spectral")
displayRate.set_limits(xlim=[surx_min,surx_max],ylim=[sury_min,sury_max])
displayRate.plot_range_rings([30, 60])#, 150, 200, 250])
displayRate.plot_cross_hair(2.)
# displayRate.set_limits(xlim=(-300,300),ylim=(-300,300))



#ZDR (input)

axZdr = figRate.add_subplot(223)
displayRate.plot_ppi('ZDR_ATTEN_CORRECTED', sur_sweep,cmap='pyart_HomeyerRainbow',vmin=-1,vmax=4,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="Zdr [dB]",
    mask_outside=True)
displayRate.plot_range_rings([30, 60])#, 150, 200, 250])
displayRate.plot_cross_hair(2.)
displayRate.set_limits(xlim=[surx_min,surx_max],ylim=[sury_min,sury_max])

# RATE_HYBRID (computed)

axHybrid = figRate.add_subplot(224)
displayRate.plot_ppi('RHOHV', sur_sweep, vmin=0.8,vmax=1.0,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="rho$_{hv}$",
    cmap = "Reds")
displayRate.plot_range_rings([30, 60])#, 150, 200, 250])
displayRate.plot_cross_hair(2.)
displayRate.set_limits(xlim=[surx_min,surx_max],ylim=[sury_min,sury_max])


figRate.tight_layout()

plt.show()


#### Now let's go back and zoom in on the SE quadrant where the most intense convection is, and we have an RHI scan. Uncomment box 11 and rerun cells 12 and 13.

## Now let's plot a vertical slice from an RHI in this case

In [ ]:
rhix_min=0
rhix_max=65
rhiy_min=0
rhiy_max=18

rhi_az= 1

In [ ]:
# Plot the RHI

displayRate = pyart.graph.RadarDisplay(rhi_seapol)
figRate = plt.figure(1, (12, 6))

# DBZ (input)

axDbz = figRate.add_subplot(121)
displayRate.plot_rhi('DBZ_ATTEN_CORRECTED', rhi_az, vmin=0, vmax=80.,
                    axislabels=("x(km)", "y(km)"),cmap='pyart_NWSRef',
                    colorbar_label="DBZ")
displayRate.set_limits(xlim=[rhix_min,rhix_max],ylim=[rhiy_min,rhiy_max])
#displayRate.set_limits(xlim=(-70,70),ylim=(-70,70))

# VEL(input)

axDbz = figRate.add_subplot(122)
displayRate.plot_rhi('VEL', rhi_az, vmin=-20, vmax=20.,
                    axislabels=("x(km)", "y(km)"),cmap='pyart_NWSVel',
                    colorbar_label="VR (m/s)")
displayRate.set_limits(xlim=[rhix_min,rhix_max],ylim=[rhiy_min,rhiy_max])
#displayRate.set_limits(xlim=(-70,70),ylim=(-70,70))



figRate.tight_layout()

plt.show()


In [ ]:
# Plot results of RadxRate

displayRate = pyart.graph.RadarDisplay(rhi_seapol)
figRate = plt.figure(1, (12, 10))

# DBZ (input)

axDbz = figRate.add_subplot(221)
displayRate.plot_rhi('DBZ_ATTEN_CORRECTED', rhi_az, vmin=0, vmax=80.,
                    axislabels=("x(km)", "y(km)"),cmap='pyart_NWSRef',
                    colorbar_label="DBZ")
displayRate.set_limits(xlim=[rhix_min,rhix_max],ylim=[rhiy_min,rhiy_max])

# KDP (computed)

axKdp = figRate.add_subplot(222)
displayRate.plot_rhi('KDP', rhi_az, vmin=0, vmax=5.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="KDP (deg/km)",
    cmap="nipy_spectral")
displayRate.set_limits(xlim=[rhix_min,rhix_max],ylim=[rhiy_min,rhiy_max])


#ZDR (input)

axZdr = figRate.add_subplot(223)
displayRate.plot_rhi('ZDR_ATTEN_CORRECTED', rhi_az,cmap='pyart_HomeyerRainbow',vmin=-1,vmax=4,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="Zdr [dB]",
    mask_outside=True)
displayRate.set_limits(xlim=[rhix_min,rhix_max],ylim=[rhiy_min,rhiy_max])

# RATE_HYBRID (computed)

axHybrid = figRate.add_subplot(224)
displayRate.plot_rhi('RHOHV', rhi_az, vmin=0.8,vmax=1.0,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="rho$_{hv}$",
    cmap = "Reds")
displayRate.set_limits(xlim=[rhix_min,rhix_max],ylim=[rhiy_min,rhiy_max])


figRate.tight_layout()

plt.show()


#### Other RHI angles were taken through the storm. Check out other slices by changing rhi_az in box 14 to other values ranging from 0 to 8 and re run cells 15 and 16.

# Hydrometeor, or Particle Identification (HID / PID / HCA)

We can infer certain things about the combination of these polarimetric signatures. For example, large Zdr and large Kdp mean there is a lot of water content and some large drops suggesting heavy rainfall, while near zero Zdr with large Z is indicative of hail. Low Rhohv suggests hydrometer diversity within a volume, and high Zdr aloft is usually a signature of oriented ice crystals.

This information is combined in particle identification algorithms, often using fuzzy logic. In fuzzy logic, different ranges of variables are defined for different hydrometeor types. The data are then scored based on how well the fit the hydrometeor ranges, and the top score is then assigned to each grid point. More detail can be found in the LROSE wiki: 

http://wiki.lrose.net/index.php/RadxPid_fuzzylogic

In the NCAR PID which is used in LROSE RadxRate_Pid, 2D membership functions define 17 categories, including non-meteorological scatterers (from Vivekanandan et al. 1999)

Vivekanandan, J., Zrnic, D. S., Ellis, S. M., Oye, R., Ryzhkov, A. V., & Straka, J. (1999). Cloud Microphysics Retrieval Using S-Band Dual-Polarization Radar Measurements, Bulletin of the American Meteorological Society, 80(3), 381-388.

<div>
<img src="../../ams2023/images/NCAR_PID.png" width="600"/>
</div>


Of course, temperature is an important part of any identification scheme. Here we are using a model-derived temperature profile over the SEA-POL site. Temperature is most important for determining the melting layer.


We can create a colormap for visualizing PID.

In [ ]:
pidmap = np.array([[0.12156862745098039, 0.46666666666666667, 0.70588235294117652, 1.0],
              [0.68235294117647061, 0.7803921568627451, 0.90980392156862744, 1.0],
              [0.59607843137254901, 0.87450980392156863, 0.54117647058823526, 1.0],
              [0.45490196078431372, 0.7686274509803922, 0.46274509803921571, 1.0],
              [0.17254901960784313, 0.62745098039215685, 0.17254901960784313, 1.0],
              [0.83921568627450982, 0.15294117647058825, 0.15686274509803921, 1.0],
              [1.0, 0.59607843137254901, 0.58823529411764708, 1.0],
              [1.0, 0.49803921568627452, 0.054901960784313725, 1.0],
              [1.0, 0.73333333333333328, 0.47058823529411764, 1.0],
              [0.61960784313725492, 0.85490196078431369, 0.89803921568627454, 1.0],
              [0.090196078431372548, 0.74509803921568629, 0.81176470588235294, 1.0],
              [0.61176470588235299, 0.61960784313725492, 0.87058823529411766, 1.0],
              [0.32156862745098042, 0.32941176470588235, 0.63921568627450975, 1.0],
              [0.859375, 0.859375, 0.859375, 1.0],
              [0.66015625, 0.66015625, 0.66015625, 1.0],
              [0.41015625, 0.41015625, 0.41015625, 1.0],
              [0.0, 0.0, 0.0, 1.0],],'f')
my_cmap2 = colors.ListedColormap(pidmap, name='ncar_pid')


In [ ]:
# Plot results of RadxRate

displayRate = pyart.graph.RadarDisplay(rhi_seapol)
figRate = plt.figure(1, (12, 10))

# DBZ (input)

swp = 1

axDbz = figRate.add_subplot(221)
displayRate.plot_rhi('DBZ_ATTEN_CORRECTED', swp, vmin=0, vmax=80.,
                    axislabels=("x(km)", "y(km)"),cmap='pyart_NWSRef',
                    colorbar_label="DBZ")
displayRate.set_limits(xlim=[0,60],ylim=[0,18])
#displayRate.plot_range_rings([30, 60])#, 150, 200, 250])
displayRate.plot_cross_hair(2.)
#displayRate.set_limits(xlim=(-300,300),ylim=(-300,300))

# KDP (computed)

axKdp = figRate.add_subplot(222)
displayRate.plot_rhi('KDP', swp, vmin=0, vmax=5.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="KDP (deg/km)",
    cmap="nipy_spectral")
displayRate.set_limits(xlim=[0,60],ylim=[0,18])
#displayRate.plot_range_rings([30, 60])#, 150, 200, 250])
displayRate.plot_cross_hair(2.)
# displayRate.set_limits(xlim=(-300,300),ylim=(-300,300))


axZdr = figRate.add_subplot(223)
displayRate.plot_rhi('ZDR_ATTEN_CORRECTED', swp, vmin=-1, vmax = 5,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="Zdr [dB]",
    cmap = 'pyart_HomeyerRainbow', mask_outside=True)


#'pyart_SCook18
#displayRate.plot_range_rings([30, 60])#, 150, 200, 250])
displayRate.plot_cross_hair(2.)
displayRate.set_limits(xlim=[0,60],ylim=[0,18])
#displayRate.set_limits(xlim=[surx_min,surx_max],ylim=[sury_min,sury_max])


# NCAR PID (computed)

axPID = figRate.add_subplot(224)
displayRate.plot_rhi('PID', swp, vmin=0.5, vmax = 17.5,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="PID",
    cmap = my_cmap2, mask_outside=True)
#    cmap = 'pyart_PD17', mask_outside=True)
#displayRate.plot_range_rings([30, 60])#, 150, 200, 250])
displayRate.plot_cross_hair(2.)
displayRate.set_limits(xlim=[0,60],ylim=[0,18])

# plot all 17 PID categories
pid_cbar = displayRate.cbs[3]
#pid_cbar.set_ticks([1,2,3,4,5,6,7,8,9,10,11,12,13])
#pid_cbar.set_ticklabels(['cld-drops', 'drizzle', 'lt-rain', 'mod-rain', 'hvy-rain', 'hail', 'rain/hail', 'sm-hail', 'gr/rain', 'dry-snow', 'wet-snow', 'ice', 'irreg-ice'])
pid_cbar.set_ticks([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])
pid_cbar.set_ticklabels(['cld-drops', 'drizzle', 'lt-rain', 'mod-rain', 'hvy-rain', 'hail', 'rain/hail', 'sm-hail', 'gr/rain', 'dry-snow', 'wet-snow', 'ice', 'irreg-ice', 'slw', 'insects', '2nd-trip', 'clutter'])



figRate.tight_layout()

plt.show()


# Rain Rate estimation

Finally, we can leverage the information provided by the different polarimetric variables to estimate the rain rate. There are many different polarimetric rain estimators, including R-Z, R-Kdp, R-Zdr-Kdp, R-Zdr, R-A (attenuation). These are a balance of the noise and uncertainty of each measurement and their strength. For example, Kdp is directly proportional to the liquid water content and insensitive to hail, but it is noisy at small values. Zdr can provide information about the size of particles in a volume. So, blended or hybrid algorithms combine these estimators in a logical sense based on the variable quality and what is in the radar volume.  For more information, see the LROSE wiki:

http://wiki.lrose.net/index.php/RadxRate_equations#NCAR_Hybrid

Let's look at what a Simple Z-R gives compared to a polarimetric blended estimator.

In [ ]:
#Reset our plotting parameters after playing around with them above.

sur_sweep =3

surx_min= 15
surx_max = 55
sury_min = -55 
sury_max = -15

In [ ]:
sur_seapol.fields.keys()

In [ ]:
# Plot results of RadxRate

displayRate = pyart.graph.RadarDisplay(sur_seapol)
figRate = plt.figure(1, (12, 10))

# DBZ (input)

axDbz = figRate.add_subplot(221)
displayRate.plot_ppi('DBZ_ATTEN_CORRECTED', sur_sweep, vmin=0, vmax=80.,
                    axislabels=("x(km)", "y(km)"),cmap='pyart_NWSRef',
                    colorbar_label="DBZ")
displayRate.set_limits(xlim=[surx_min,surx_max],ylim=[sury_min,sury_max])
displayRate.plot_range_rings([30, 60])#, 150, 200, 250])
displayRate.plot_cross_hair(2.)
#displayRate.set_limits(xlim=(-70,70),ylim=(-70,70))

# KDP (computed)

axKdp = figRate.add_subplot(222)
displayRate.plot_ppi('RATE_KDP', sur_sweep, norm=colors.LogNorm(vmin=0.1, vmax=150),
    axislabels=("x(km)", "y(km)"),
    colorbar_label="RATE_KDP (mm/hr)",
    cmap = "pyart_RRate11")
displayRate.set_limits(xlim=[surx_min,surx_max],ylim=[sury_min,sury_max])
displayRate.plot_range_rings([30, 60])#, 150, 200, 250])
displayRate.plot_cross_hair(2.)
# displayRate.set_limits(xlim=(-300,300),ylim=(-300,300))



# KDP (computed)

axKdp = figRate.add_subplot(223)
displayRate.plot_ppi('KDP', sur_sweep, vmin=0, vmax=5.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="KDP (deg/km)",
    cmap="nipy_spectral")
displayRate.set_limits(xlim=[surx_min,surx_max],ylim=[sury_min,sury_max])
displayRate.plot_range_rings([30, 60])#, 150, 200, 250])
displayRate.plot_cross_hair(2.)
# displayRate.set_limits(xlim=(-300,300),ylim=(-300,300))


# RATE_HYBRID (computed)

axHybrid = figRate.add_subplot(224)
displayRate.plot_ppi('RATE_HYBRID', sur_sweep, norm=colors.LogNorm(vmin=0.1, vmax=150),
    axislabels=("x(km)", "y(km)"),
    colorbar_label="RATE_HYBRID(mm/hr)",
    cmap = "pyart_RRate11")
displayRate.plot_range_rings([30, 60])#, 150, 200, 250])
displayRate.plot_cross_hair(2.)
displayRate.set_limits(xlim=[surx_min,surx_max],ylim=[sury_min,sury_max])


figRate.tight_layout()

plt.show()


# 5. Extras

<a id=’RadxRate-parameters’></a>
## **RadxRate parameters**

Below is a list of important RadxRate parameters. The line numbers are accurate as of Dec 2022 for the final Topaz release of LROSE. 

* **RadxRate main parameters**
    * input_dir [line 105]: path to input data, can be specified on command line
    * INPUT FIELD INFORMATION [line 155]: must match input file variable names
        * here SNR_available = FALSE [line 177]
        * here DBZ_field_name = "REF" [line 215]
        * here LDR_available = FALSE [line 255]
    * kdp_params_file_path [line 284]: path to Kdp-specific parameter file
    * pid_params_file_path [line 303]: path to PID-specific parameter file
    * PID_use_attenuation_corrected_fields [line 331]: specify whether to use attenuation-corrected DBZ and ZDR in PID
    * RATE_params_file_path [line 350]: path to precipitation rate coefficients parameter file
    * RATE_use_attenuation_corrected_fields [line 378]: specify whether to use attenuation-corrected DBZ and ZDR in precipitation rate calculation
    * output_dir [line 699]: path where output files are written, can set on command line with -outdir
    * output_format [line 755]: output file format, usually CfRadial
* **RadxRate Kdp_specific parameters**
    * KDP_fir_filter_len [line 65]: filter length used for KDP calculation
        * here KDP_FIR_LEN_10
    * KDP_psob_method [line 114]: specify method to remove phase shift on backscatter
        * here PEAK_REMOVAL_METHOD
* **RadxRate Pid_specific parameters**
    * PID_thresholds_file_path [line 27]: path to fuzzy logic PID thresholds file
    * PID_use_soundings_from_spdb [line 303]: specify whether soundings are in Spdb format, otherwise sounding found in fuzzy logic file
        * here TRUE
    * PID_sounding_spdb_url [line 313]: path to Spdb soundings
    * PID_sounding_location_name [line 339]: name of sounding location
        * here $(RADAR_NAME)
* **RadxRate Rate_specific parameters**
    * RATE_zh_aa [line 154]: R(Z) coefficients
    * RATE_zh_bb [line 164]: R(Z) coefficients
    * RATE_zh_aa_snow [line 182]: R(Z) coefficients in snow
    * RATE_zh_bb_snow [line 192]: R(Z) coefficients in snow
    * RATE_zzdr_aa [line 210]: R(Z, ZDR) coefficients
    * RATE_zzdr_bb [line 220]: R(Z, ZDR) coefficients
    * RATE_zzdr_cc [line 230]: R(Z, ZDR) coefficients
    * RATE_kdp_aa [line 248]: R(KDP) coefficients
    * RATE_kdp_bb [line 258]: R(KDP) coefficients
    * RATE_kdpzdr_aa [line 277]: R(KDP, ZDR) coefficients
    * RATE_kdpzdr_bb [line 287]: R(KDP, ZDR) coefficients
    * RATE_kdpzdr_cc [line 297]: R(KDP, ZDR) coefficients
* **Pid Thresholds parameters**
    * Radar dependent
        * SEA-POL is C-band simultaneous h/v
        * NEXRAD is S-band simultaneous h/v transmission and recieving
        * Other parameter files can be found [here](http://wiki.lrose.net/index.php/RadxPid#PID_thresholds_file)
